In [ ]:
''''''
# Pedestrian volume anomaly detection
# Author:   Amir Rafe (amir.rafe@usu.edu)
# File:     PedAnomaly.ipynb
# Date:     Summer 2023
# Version:  1.21  
# About:    Methods to detect anomalies amont time series pedestrian volume data
''''''

In [ ]:
## Loading packages

import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
#
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.seasonal import seasonal_decompose
#
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
#
import plotly.graph_objects as go
import plotly.express as px
#
from scipy import stats
import ruptures as rpt
#
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from pyod.models.knn import KNN
#
import torch
import torch.nn as nn
#
from sklearn.preprocessing import MinMaxScaler
#
from darts.models import RNNModel, TCNModel
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import GaussianLikelihood

# Data preparation

In [ ]:
## Load and preparing data

# Load sample data
df= pd.read_csv ('5323.csv')
df['TIME1'] = pd.to_datetime(df['TIME1'])

# Filter data based on COVID-19 phases in UTAH

# Phase zero: pre-COVID19
dfph0 = df.loc[(df['TIME1'] <= '2020-03-05')]

# Phase one between March 2020 and April 2020
start_date = pd.to_datetime('2020-03-06')
end_date = pd.to_datetime('2020-04-30')
dfph1 = df.loc[(df['TIME1'] >= start_date) & (df['TIME1'] <= end_date)]

# Phase two between May 2020 and May 2021
start_date = pd.to_datetime('2020-05-01')
end_date = pd.to_datetime('2021-05-31')
dfph2 = df.loc[(df['TIME1'] >= start_date) & (df['TIME1'] <= end_date )]

# Phase three between June 2021 and Feburary 2022
start_date = pd.to_datetime('2021-06-01')
end_date = pd.to_datetime('2022-02-28')
dfph3 = df.loc[(df['TIME1'] >= start_date) & (df['TIME1'] <= end_date)]

# Phase four after March 2022
dfph4 = df.loc[(df['TIME1'] >= '2022-03-01')]

# Monthly data (choose start and end date based on your analysis type)
start_date = pd.to_datetime('2022-03-06')
end_date = pd.to_datetime('2022-04-06')
dfM = df.loc[(df['TIME1'] >= start_date) & (df['TIME1'] <= end_date)]

# Subset data for analysis
data  = df[['TIME1' , 'PED', 'High Temp' , 'Low Temp', 'Precipitation' , 'Daily Case Rate / 100,000' , 'AQI']] 
data.set_index('TIME1', inplace=True)
data = data.copy()
data.loc[:, 'Average Temp'] = (data.loc[:, 'High Temp'] + data.loc[:, 'Low Temp']) / 2

# Drop 'High Temp' and 'Low Temp' columns
data = data.drop(['High Temp', 'Low Temp'], axis=1)

In [ ]:
## Decomposition, Seasonality and Stationarity

df_clean = dfM.copy() # choose the desired subdatset
df_clean = df_clean.dropna()

# Perform the decomposition
decomposition = seasonal_decompose(df_clean['PED'], period=12)
decomposition.plot()
plt.show()

# Seasonality check
autocorrelation_plot(df_clean['PED'])
plt.title('Autocorrelation of Pedestrian Volume')
plt.show()

# Check stationarity of the variables 
dataNa = data.dropna()
def check_stationarity(data):
    stationary = True
    for col in data.columns:
        result = adfuller(data[col])
        p_value = result[1]
        if p_value > 0.05:
            print(f"Warning: {col} is non-stationary (p-value = {p_value})")
            stationary = False
    return stationary
check_stationarity(dataNa)

# Detect Change points and VAR analysis

In [ ]:
## Detect change points 

# Choose a method from the following lists:

    #'Least absolute deviation': 'l1'
    #'Least squared deviation_2': 'l2',
    #'Gaussian process change': 'normal',
    #'Kernelized mean change': 'rbf',
    #'Autoregressive model change': 'ar'
    #rpt.BottomUp
    #rpt.Window
    #rpt.Dynp
    #rpt.Binseg

data1  = df[['TIME1' , 'PED']]
data1.set_index('TIME1', inplace=True)
algo = rpt.Binseg(model="normal").fit(data1)
result = algo.predict(n_bkps=15)

fig = go.Figure()
fig.add_trace(go.Scatter(x=data1.index, y=data1['PED'], mode='lines', name='Pedestrian'))
for cp in result:
    if cp < len(data1.index) - 1:
        fig.add_shape(type='line', x0=data1.index[cp], y0=0, x1=data1.index[cp], y1=max(data1['PED']), line=dict(color='red', width=1))

change_point_dates = []
for cp in result:
    if cp < len(data1.index) - 1:
        change_point_date = data1.index[cp]
        change_point_dates.append(change_point_date)
        fig.add_shape(type='line', x0=change_point_date, y0=0, x1=change_point_date, y1=max(data1['PED']), line=dict(color='red', width=1))

print("Change Point Dates:")
print(change_point_dates)
fig.show()

In [ ]:
## VAR analysis between each change points and craete AD (Analysis Duration)

# RMSE function
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Data frame start and end point
start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2022-12-31')

# Create subset dataframe between change points
subset_dataframes = []
previous_date = start_date

for change_point_date in change_point_dates:
    if change_point_date <= start_date or change_point_date >= end_date:
        continue
    
    # Check if the time difference is less than one month
    if (change_point_date - previous_date).days > 30:
        subset_data = data.loc[previous_date:change_point_date - pd.DateOffset(days=1)]
        subset_dataframes.append(subset_data)
        previous_date = change_point_date

# Add the last subset from the last change point to the end of the data
if previous_date < end_date:
    subset_data = data.loc[previous_date:end_date]
    subset_dataframes.append(subset_data)

# VAR analysis
for i, subset_data in enumerate(subset_dataframes):
    print(f"Subset {i+1}:")
    
    # Get the start date of the subset
    start_date = subset_data.index[0]

    # Set a specific date range for the subset data
    subset_data.index = pd.date_range(start=start_date, periods=len(subset_data), freq='D')

    # Remove rows with missing values
    subset_data = subset_data.dropna()

    # Difference the data
    subset_data_diff = subset_data - subset_data.shift(5) # based on seasonality
    subset_data_diff = subset_data_diff.dropna()

    # Split the data into training and testing sets (using an 80-20 split)
    train_size = int(len(subset_data_diff) * 0.8)
    train_data = subset_data_diff[:train_size]
    test_data = subset_data_diff[train_size:]

    # Fit the VAR model
    model = VAR(train_data)

    try:
        # Determine the optimal lag order using AIC or BIC
        lag_order = model.select_order()
    except KeyError:
        print("Error: Could not calculate AIC for model order selection.")
        continue

    # Fit the VAR model with the chosen lag order
    results = model.fit(lag_order.aic)

    # Print the summary of the model
    print(results.summary())

    # Evaluate the model on the testing data
    forecast = results.forecast(y=train_data.values, steps=len(test_data))
    forecast_df = pd.DataFrame(forecast, index=test_data.index, columns=test_data.columns)

    # Calculate RMSE for each variable
    for col in forecast_df.columns:
        print(f"RMSE for {col}: {rmse(forecast_df[col], test_data[col])}")

    print(check_stationarity(subset_data_diff))
    
    # Get the p-values for the PED variable
    p_values = results.pvalues['PED']

    # Set the significance level
    alpha = 0.05

    # List of variable names
    variables = ['Daily Case Rate / 100,000', 'AQI', 'Average Temp', 'Precipitation']

    # Loop through each variable
    for variable in variables:
        
        # Find the significant variables and their lags
        significant_vars = []
        for lag in range(1, lag_order.aic + 1):
            if p_values[f'L{lag}.{variable}'] < alpha:
                significant_vars.append((f'L{lag}.{variable}', lag))

        # Print the significant variables and their lags
        if significant_vars:
            print("Significant variables and their lags:")
            for var, lag in significant_vars:
                print(f"{var} at lag {lag}")
        else:
            print(f"No significant variables found for the PED equation with {variable}.")

print()


# Inject synthetic data

In [ ]:
## Inject synthetic outliers

# Create a copy of the original DataFrame to store the outliers
data1 = data1.sort_index()
data_outliers = data1.copy()

# Add a column 'is_outlier' initialized with zeros
data_outliers['is_outlier'] = 0

# Options to generate steady outliers
#random.seed(42)
#np.random.seed(42)

# Store dates of the change points, adding the first and the last date of the dataframe
# Also filter out any invalid indices from the result list
result_filtered = [i for i in result if i < len(data1)]
changepoints = [data1.index[0]] + [data1.index[i] for i in result_filtered] + [data1.index[-1]]

# Define outliers in each segment
num_outliers_per_segment = random.choice([2, 5])

# Define the range of increase or decrease the values by percentage
percentage_change = 0.60 
percentage_change2 = 0.85

# For each pair of change points
for i in range(len(changepoints) - 1):
    # Check if the period is between June 15, 2019 and September 8, 2019 (period of the block missing value for the sample data)
    if (changepoints[i] > pd.to_datetime('2019-06-15') and 
        changepoints[i+1] < pd.to_datetime('2019-09-08')):
        continue

    # Generate outliers
    for _ in range(num_outliers_per_segment):
        # Select a random date between the current and next change point
        outlier_date = np.random.choice(pd.date_range(changepoints[i], changepoints[i+1]))

        # Generate an outlier by increasing or decreasing the value at the chosen date by a specific percentage
        increase_or_decrease = 1 + 1 * random.choice([percentage_change, percentage_change2])
        outlier = data_outliers.loc[outlier_date, 'PED'] * increase_or_decrease

        # Replace the value at the chosen date with the generated outlier in the outlier DataFrame
        data_outliers.loc[outlier_date, 'PED'] = outlier

        # Mark the generated outlier in 'is_outlier' column
        data_outliers.loc[outlier_date, 'is_outlier'] = 1

# Create a separate DataFrame for outliers by filtering only the modified rows
data_only_outliers = data_outliers[data_outliers['is_outlier'] == 1]

fig = go.Figure()

# Add pedestrian volume to the plot
fig.add_trace(go.Scatter(x=data_outliers.index, y=data_outliers['PED'], mode='lines', name='Data with injected outliers'))
fig.add_trace(go.Scatter(x=data1.index, y=data1['PED'], mode='lines', name='Original data'))
# Add outliers to the plot
fig.add_trace(go.Scatter(x=data_only_outliers.index, y=data_only_outliers['PED'], mode='markers', name='Synthetic outliers', marker=dict(color='orange', size=10)))
fig.update_layout(
                  xaxis=dict(title='Date'),
                yaxis=dict(title='Pedestrian volume'))
# Add change points to the plot
for cp in result_filtered:
    fig.add_shape(type='line', x0=data1.index[cp], y0=0, x1=data1.index[cp], y1=max(data_outliers['PED']), line=dict(color='gray', width=1))

fig.show()

# Anomaly detection methods

In [ ]:
## Isolation Forest method

# Drop rows with NaN values
data_outliers = data_outliers.dropna()

# Reshape the values column to 2D array as needed by the model
values = data_outliers['PED'].values.reshape(-1, 1)

# Define the model
model = IsolationForest(contamination=0.1)

# Fit the model
model.fit(values)

# Predict the outliers
outliers = model.predict(values)

# Replace outliers (-1) with NaN
data_outliers['PED'][outliers == -1] = np.nan

# Reset index to plot data using Plotly 
data_outliers_reset = data_outliers.reset_index()

# Plot the final dataset
fig = px.line(data_outliers_reset, x='TIME1', y='PED', title='Time Series with Outliers Replaced by NaN')
fig.add_trace(go.Scatter(x=data1.index, y=data1['PED'], mode='lines', name='Data with injected outliers'))
fig.show()


In [ ]:
# KNN method

# Replace initial NaN values with a placeholder value
data_outliers.fillna(-99999, inplace=True)

# Create a 2D array of the 'PED' column values
values = data_outliers['PED'].values.reshape(-1,1)

# Define the model
clf = KNN(contamination=0.1)

# Fit the model
clf.fit(values)

# Get the prediction labels of the training data
y_train_pred = clf.labels_ 

# Outlier points are predicted as 1 while inliers are predicted as 0
data_outliers['Outlier'] = y_train_pred

# Replace outlier points and initial NaN values (placeholder values) with NaN
data_outliers.loc[data_outliers['Outlier'] == 1, 'PED'] = np.nan
data_outliers.replace(-99999, np.nan, inplace=True)

# Drop the 'Outlier' column
data_outliers.drop('Outlier', axis=1, inplace=True)

# Create the initial line trace
fig = go.Figure()

# Add outliers to the plot
fig.add_trace(go.Scatter(x=data_only_outliers.index, y=data_only_outliers['PED'], mode='markers', name='Synthetic outliers', marker=dict(color='orange', size=10)))
fig.add_trace(go.Scatter(x=data_outliers_reset['TIME1'], y=data_outliers_reset['PED'], mode='lines', name='Cleaned data', line=dict(color='#636EFA'))) 

for cp in result_filtered:
    # Add a line with go.Scatter
    fig.add_trace(go.Scatter(x=[data1.index[cp], data1.index[cp]], y=[0, max(data_outliers['PED'])], mode='lines', line=dict(color='gray', width=1), showlegend=False))

# Use the update_layout() function to update axis titles
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Pedestrian volume',
)

fig.show()


In [ ]:
## DBSCAN method

# Drop rows with NaN values
data_outlierss = data_outliers.dropna()

# Reshape the values column to 2D array as needed by the model
values = data_outlierss['PED'].values.reshape(-1, 1)

# Define the model
model = DBSCAN(min_samples=11, eps=0.5)

# Fit the model and predict outliers
outliers = model.fit_predict(values)

# Replace outliers (-1) with NaN
data_outlierss['PED'][outliers == -1] = np.nan

# Reset index to plot data using Plotly 
data_outliers_reset = data_outlierss.reset_index()

# Create the initial line trace
fig = go.Figure()

# Add outliers to the plot
fig.add_trace(go.Scatter(x=data_only_outliers.index, y=data_only_outliers['PED'], mode='markers', name='Synthetic outliers', marker=dict(color='orange', size=10)))
fig.add_trace(go.Scatter(x=data_outliers_reset['TIME1'], y=data_outliers_reset['PED'], mode='lines', name='Cleaned data', line=dict(color='#636EFA'))) 

for cp in result_filtered:
    # Add a line with go.Scatter
    fig.add_trace(go.Scatter(x=[data1.index[cp], data1.index[cp]], y=[0, max(data_outliers['PED'])], mode='lines', line=dict(color='gray', width=1), showlegend=False))

# Use the update_layout() function to update axis titles
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Pedestrian volume',
)

fig.show()

# Imputation methods

In [ ]:
## Random Forest method

# Create a mask for the period we are interested in
mask = (data_outlierss.index > '2018-01-01') & (data_outlierss.index <= '2022-12-31')

# Create a new DataFrame for that period
df_period = data_outlierss.loc[mask].copy()

# Convert date index to ordinal for training purpose
df_period['date_ordinal'] = df_period.index.map(dt.toordinal)

# Prepare data for imputation
X = df_period[['date_ordinal', 'PED']].values

# Split data into data with known 'PED' and data with unknown 'PED'
known = X[~np.isnan(X[:, 1])]  # rows where 'PED' is known
unknown = X[np.isnan(X[:, 1])]  # rows where 'PED' is unknown

# Create RandomForestRegressor
regressor = RandomForestRegressor(random_state=0, n_estimators=100)

# Fit the regressor and predict
if unknown.size > 0:  # If there are any unknown 'PED' values
    regressor.fit(known[:, 0].reshape(-1, 1), known[:, 1])  # Fit the regressor
    predictions = regressor.predict(unknown[:, 0].reshape(-1, 1))  # Predict the unknown 'PED' values
    X[np.isnan(X[:, 1]), 1] = predictions  # Fill the unknown 'PED' values with the predictions

# Update 'PED' in df_period with imputed data
df_period['PED'] = X[:, 1]

# Plot the original data and imputed data
fig = go.Figure()

# Imputed data
fig.add_trace(go.Scatter(
    x=df_period.index,
    y=df_period['PED'],
    mode='lines',
    name='Imputed data',
    line=dict(color='red')
))

# Original data
fig.add_trace(go.Scatter(
    x=data_outlierss.index,
    y=data_outlierss['PED'],
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False
))

# Update x-axis range
#fig.update_xaxes(range=['2018-08-21', '2019-04-01'])

# Use the update_layout() function to update axis titles
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Pedestrian volume',
)

fig.show()


In [ ]:
## KNN method

# Create a mask for the period we are interested in
mask = (data_outlierss.index > '2018-01-01') & (data_outlierss.index <= '2022-12-31')

# Create a new DataFrame for that period
df_period = data_outlierss.loc[mask]

# Convert date index to ordinal for training purpose
df_period['date_ordinal'] = df_period.index.map(dt.toordinal)

# Prepare data for imputation
X = df_period[['date_ordinal', 'PED']].values

# Create KNN imputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")

# Perform imputation
X_imputed = imputer.fit_transform(X)

# Update 'value' in df_period with imputed data
df_period['PED'] = X_imputed[:, 1]

# Plot the original data and imputed data
fig = go.Figure()

# Imputed data
fig.add_trace(go.Scatter(
    x=df_period.index,
    y=df_period['PED'],
    mode='lines',
    name='Imputed data',
    line=dict(color='red')
))

# Original data
fig.add_trace(go.Scatter(
    x=data_outlierss.index,
    y=data_outlierss['PED'],
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False
))
# Update x-axis range
#fig.update_xaxes(range=['2018-08-21', '2019-04-01'])

# Use the update_layout() function to update axis titles
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Pedestrian volume',
)


fig.show()


In [ ]:
## LSTM method - Univariate imputation

# Define LSTM Model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
# Prepare data
data1 = data.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
scaler = MinMaxScaler(feature_range=(-1, 1))
data_normalized = scaler.fit_transform(data1['PED'].values.reshape(-1, 1))

# Transform data to tensors
data_normalized = torch.FloatTensor(data_normalized).view(-1)

# Define a method to create in/out sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Prepare sequences
train_window = 20  
train_inout_seq = create_inout_sequences(data_normalized, train_window)

# Initialize the model, define loss function and optimizer
model = LSTM(1, 20, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

# Make predictions
model.eval()
fut_pred = 91
test_inputs = data_normalized[-train_window:].tolist()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

# Invert normalization
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:]).reshape(-1, 1))

# Plot the actual vs predicted data
plt.plot(data1['PED'].values, label='actual')
plt.plot(range(len(data1['PED']), len(data1['PED']) + fut_pred), actual_predictions, label='forecast')
plt.legend()


In [ ]:
## LSTM method - Multivariate imputation

class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Prepare data
scaler = MinMaxScaler(feature_range=(-1, 1))

# Include the time series and exogenous variables
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data1 = data1[['PED', 'AQI', #'Daily Case Rate / 100,000'
               'Average Temp', 'Precipitation']]

# Normalize the dataset
data_normalized = scaler.fit_transform(data1)

# Transform data to tensors
data_normalized = torch.FloatTensor(data_normalized)

# Define a method to create in/out sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1, 0]  
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Prepare sequences
train_window = 20  
train_inout_seq = create_inout_sequences(data_normalized, train_window)

# Initialize the model, define loss function and optimizer
model = LSTM(4, 20, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

# Make predictions
model.eval()
fut_pred = 91
# Take the last `train_window` sets of data points
test_inputs = data_normalized[-train_window:].tolist()
test_inputs = test_inputs + [[0]*4 for _ in range(fut_pred)]

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[i:i+train_window])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs[i+train_window][0] = model(seq).item()  
        
# Extract the `fut_pred` predictions made by the model
predictions = test_inputs[-fut_pred:]

# Invert normalization
actual_predictions = scaler.inverse_transform(predictions)[:, 0]  

# Plot the actual vs predicted data
plt.plot(data1['PED'].values, label='actual')
plt.plot(range(len(data1['PED']), len(data1['PED']) + fut_pred), actual_predictions, label='forecast')
plt.legend()


In [ ]:
## GRU method - Univariate imputation

# Define GRU Model
class GRU(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.gru = nn.GRU(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = torch.zeros(1,1,self.hidden_layer_size)

    def forward(self, input_seq):
        gru_out, self.hidden_cell = self.gru(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(gru_out.view(len(input_seq), -1))
        return predictions[-1]

# Prepare data
data1 = data.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
scaler = MinMaxScaler(feature_range=(-1, 1))
data_normalized = scaler.fit_transform(data1['PED'].values.reshape(-1, 1))

# Transform data to tensors
data_normalized = torch.FloatTensor(data_normalized).view(-1)

# Define a method to create in/out sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Prepare sequences
train_window = 20  
train_inout_seq = create_inout_sequences(data_normalized, train_window)

# Initialize the model, define loss function and optimizer
model = GRU(1, 20, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = torch.zeros(1, 1, model.hidden_layer_size)

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

# Make predictions
model.eval()
fut_pred = 91
test_inputs = data_normalized[-train_window:].tolist()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = torch.zeros(1, 1, model.hidden_layer_size)
        test_inputs.append(model(seq).item())

# Invert normalization
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:]).reshape(-1, 1))

# Plot the actual vs predicted data
plt.plot(data1['PED'].values, label='actual')
plt.plot(range(len(data1['PED']), len(data1['PED']) + fut_pred), actual_predictions, label='forecast')
plt.legend()


In [ ]:
## GRU method - Multivariate imputation

# Define GRU Model
class GRU(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.gru = nn.GRU(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = torch.zeros(1,1,self.hidden_layer_size)

    def forward(self, input_seq):
        gru_out, self.hidden_cell = self.gru(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(gru_out.view(len(input_seq), -1))
        return predictions[-1]

# Prepare data
scaler = MinMaxScaler(feature_range=(-1, 1))

# Include the time series and exogenous variables
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data1 = data1[['PED', 'AQI', #'Daily Case Rate / 100,000'
               'Average Temp', 'Precipitation']]

# Normalize the dataset
data_normalized = scaler.fit_transform(data1)

# Transform data to tensors
data_normalized = torch.FloatTensor(data_normalized)

# Define a method to create in/out sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1, 0]  
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Prepare sequences
train_window = 20  
train_inout_seq = create_inout_sequences(data_normalized, train_window)

# Initialize the model, define loss function and optimizer
model = GRU(4, 20, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = torch.zeros(1, 1, model.hidden_layer_size)

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

# Make predictions
model.eval()
fut_pred = 91
# Take the last `train_window` sets of data points
test_inputs = data_normalized[-train_window:].tolist()
test_inputs = test_inputs + [[0]*4 for _ in range(fut_pred)]

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[i:i+train_window])
    with torch.no_grad():
        model.hidden = torch.zeros(1, 1, model.hidden_layer_size)
        test_inputs[i+train_window][0] = model(seq).item()  

# Extract the `fut_pred` predictions made by the model
predictions = test_inputs[-fut_pred:]

# Invert normalization
actual_predictions = scaler.inverse_transform(predictions)[:, 0]  

# Plot the actual vs predicted data
plt.plot(data1['PED'].values, label='actual')
plt.plot(range(len(data1['PED']), len(data1['PED']) + fut_pred), actual_predictions, label='forecast')
plt.legend()


In [ ]:
## RNN method - Univariate imputation

# Define RNN Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.rnn = nn.RNN(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = torch.zeros(1,1,self.hidden_layer_size)

    def forward(self, input_seq):
        rnn_out, self.hidden_cell = self.rnn(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(rnn_out.view(len(input_seq), -1))
        return predictions[-1]

# Prepare data
data1 = data.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
scaler = MinMaxScaler(feature_range=(-1, 1))
data_normalized = scaler.fit_transform(data1['PED'].values.reshape(-1, 1))

# Transform data to tensors
data_normalized = torch.FloatTensor(data_normalized).view(-1)

# Define a method to create in/out sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Prepare sequences
train_window = 20  
train_inout_seq = create_inout_sequences(data_normalized, train_window)

# Initialize the model, define loss function and optimizer
model = RNN(1, 20, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = torch.zeros(1, 1, model.hidden_layer_size)

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

# Make predictions
model.eval()
fut_pred = 91
test_inputs = data_normalized[-train_window:].tolist()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = torch.zeros(1, 1, model.hidden_layer_size)
        test_inputs.append(model(seq).item())

# Invert normalization
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:]).reshape(-1, 1))

# Plot the actual vs predicted data
plt.plot(data1['PED'].values, label='actual')
plt.plot(range(len(data1['PED']), len(data1['PED']) + fut_pred), actual_predictions, label='forecast')
plt.legend()


In [ ]:
## RNN method - Multivariate imputation

# Define RNN Model
class RNN(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.rnn = nn.RNN(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = torch.zeros(1,1,self.hidden_layer_size)

    def forward(self, input_seq):
        rnn_out, self.hidden_cell = self.rnn(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(rnn_out.view(len(input_seq), -1))
        return predictions[-1]

# Prepare data
scaler = MinMaxScaler(feature_range=(-1, 1))

# Include the time series and exogenous variables
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data1 = data1[['PED', 'AQI', #'Daily Case Rate / 100,000'
               'Average Temp', 'Precipitation']]

# Normalize the dataset
data_normalized = scaler.fit_transform(data1)

# Transform data to tensors
data_normalized = torch.FloatTensor(data_normalized)

# Define a method to create in/out sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1, 0]  
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Prepare sequences
train_window = 20  
train_inout_seq = create_inout_sequences(data_normalized, train_window)

# Initialize the model, define loss function and optimizer
model = RNN(4, 20, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = torch.zeros(1, 1, model.hidden_layer_size)

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

# Make predictions
model.eval()
fut_pred = 91
# Take the last `train_window` sets of data points
test_inputs = data_normalized[-train_window:].tolist()
test_inputs = test_inputs + [[0]*4 for _ in range(fut_pred)]

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[i:i+train_window])
    with torch.no_grad():
        model.hidden = torch.zeros(1, 1, model.hidden_layer_size)
        test_inputs[i+train_window][0] = model(seq).item()  

# Extract the `fut_pred` predictions made by the model
predictions = test_inputs[-fut_pred:]

# Invert normalization
actual_predictions = scaler.inverse_transform(predictions)[:, 0]  

# Plot the actual vs predicted data
plt.plot(data1['PED'].values, label='actual')
plt.plot(range(len(data1['PED']), len(data1['PED']) + fut_pred), actual_predictions, label='forecast')
plt.legend()

In [ ]:
## LSTM method - Univariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = RNNModel(
    model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
    training_length=50,
    input_chunk_length=20,
    likelihood=GaussianLikelihood(),
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2[data2['TIME1'] >= '2019-09-10']
data2.set_index('TIME1', inplace=True)
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', 'PED')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)


# Fit the model
model.fit(data_scaled, future_covariates=covariates)

# Predict the next n days
forecast = model.predict(n=91, future_covariates=covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)

# Create a single trace for the entire dataset
trace_all = go.Scatter(
    x=np.concatenate([data_ts.time_index, forecast.time_index, data2.index]),
    y=np.concatenate([data_ts.univariate_values(), forecast.univariate_values(), data2['PED'].values]),
    mode='lines',
    name='All',
    line=dict(color='red'),
    showlegend=False
)

# Create separate traces for each segment with desired colors
trace1 = go.Scatter(
    x=data_ts.time_index, 
    y=data_ts.univariate_values(),
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

trace2 = go.Scatter(
    x=forecast.time_index, 
    y=forecast.univariate_values(),
    mode='lines',
    name='Imputed data',
    line=dict(color='red'),
)

trace3 = go.Scatter(
    x=data2.index,
    y=data2['PED'],
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

# Create the layout
layout = go.Layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Pedestrian volume'),
)

# Create the figure and add the traces
fig = go.Figure(data=[trace_all, trace1, trace2, trace3], layout=layout)

# Update x-axis range
fig.update_xaxes(range=['2019-04-10', '2019-10-30'])

# Show the figure
fig.show()

In [ ]:
## LSTM method - Multivariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = RNNModel(
    model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
    training_length=50,
    input_chunk_length=20,
    likelihood=GaussianLikelihood(),
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2.drop_duplicates(subset='TIME1')
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', ['PED', 'AQI', #'Daily Case Rate / 100,000',
                                                     'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)

# Create future covariates TimeSeries
future_covariates = TimeSeries.from_dataframe(data2, 'TIME1', ['AQI', 'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')

# Fit the model
model.fit(data_scaled, future_covariates=future_covariates)

# Predict the next n days
forecast = model.predict(n=91, future_covariates=future_covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)
forecast_PED = forecast['PED']

# Plot the forecast
data_ts['PED'].plot(label='actual')
forecast_PED.plot(label='forecast')
plt.legend()


In [ ]:
## GRU method - Univariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = RNNModel(
    model="GRU",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
    training_length=50,
    input_chunk_length=20,
    likelihood=GaussianLikelihood(),
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2[data2['TIME1'] >= '2019-09-10']
data2.set_index('TIME1', inplace=True)
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', 'PED')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)


# Fit the model
model.fit(data_scaled, future_covariates=covariates)

# Predict the next n days
forecast = model.predict(n=91, future_covariates=covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)

# Create a single trace for the entire dataset
trace_all = go.Scatter(
    x=np.concatenate([data_ts.time_index, forecast.time_index, data2.index]),
    y=np.concatenate([data_ts.univariate_values(), forecast.univariate_values(), data2['PED'].values]),
    mode='lines',
    name='All',
    line=dict(color='red'),
    showlegend=False
)

# Create separate traces for each segment with desired colors
trace1 = go.Scatter(
    x=data_ts.time_index, 
    y=data_ts.univariate_values(),
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

trace2 = go.Scatter(
    x=forecast.time_index, 
    y=forecast.univariate_values(),
    mode='lines',
    name='Imputed data',
    line=dict(color='red'),
)

trace3 = go.Scatter(
    x=data2.index,
    y=data2['PED'],
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

# Create the layout
layout = go.Layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Pedestrian volume'),
)

# Create the figure and add the traces
fig = go.Figure(data=[trace_all, trace1, trace2, trace3], layout=layout)

# Update x-axis range
fig.update_xaxes(range=['2019-04-10', '2019-10-30'])

# Show the figure
fig.show()

In [ ]:
## GRU method - Multivariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = RNNModel(
    model="GRU",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
    training_length=50,
    input_chunk_length=20,
    likelihood=GaussianLikelihood(),
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2.drop_duplicates(subset='TIME1')
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', ['PED', 'AQI', #'Daily Case Rate / 100,000',
                                                     'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)

# Create future covariates TimeSeries
future_covariates = TimeSeries.from_dataframe(data2, 'TIME1', ['AQI', 'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')

# Fit the model
model.fit(data_scaled, future_covariates=future_covariates)

# Predict the next n days
forecast = model.predict(n=91, future_covariates=future_covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)
forecast_PED = forecast['PED']

# Plot the forecast
data_ts['PED'].plot(label='actual')
forecast_PED.plot(label='forecast')
plt.legend()


In [ ]:
## RNN method - Univariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = RNNModel(
    model="RNN",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
    training_length=50,
    input_chunk_length=20,
    likelihood=GaussianLikelihood(),
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2[data2['TIME1'] >= '2019-09-10']
data2.set_index('TIME1', inplace=True)
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', 'PED')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)


# Fit the model
model.fit(data_scaled, future_covariates=covariates)

# Predict the next n days
forecast = model.predict(n=91, future_covariates=covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)

# Create a single trace for the entire dataset
trace_all = go.Scatter(
    x=np.concatenate([data_ts.time_index, forecast.time_index, data2.index]),
    y=np.concatenate([data_ts.univariate_values(), forecast.univariate_values(), data2['PED'].values]),
    mode='lines',
    name='All',
    line=dict(color='red'),
    showlegend=False
)

# Create separate traces for each segment with desired colors
trace1 = go.Scatter(
    x=data_ts.time_index, 
    y=data_ts.univariate_values(),
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

trace2 = go.Scatter(
    x=forecast.time_index, 
    y=forecast.univariate_values(),
    mode='lines',
    name='Imputed data',
    line=dict(color='red'),
)

trace3 = go.Scatter(
    x=data2.index,
    y=data2['PED'],
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

# Create the layout
layout = go.Layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Pedestrian volume'),
)

# Create the figure and add the traces
fig = go.Figure(data=[trace_all, trace1, trace2, trace3], layout=layout)

# Update x-axis range
fig.update_xaxes(range=['2019-04-10', '2019-10-30'])

# Show the figure
fig.show()

In [ ]:
## RNN method - Multivariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = RNNModel(
    model="RNN",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3},
    random_state=0,
    training_length=50,
    input_chunk_length=20,
    likelihood=GaussianLikelihood(),
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2.drop_duplicates(subset='TIME1')
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', ['PED', 'AQI', #'Daily Case Rate / 100,000',
                                                     'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)

# Create future covariates TimeSeries
future_covariates = TimeSeries.from_dataframe(data2, 'TIME1', ['AQI', 'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')

# Fit the model
model.fit(data_scaled, future_covariates=future_covariates)

# Predict the next n days
forecast = model.predict(n=91, future_covariates=future_covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)
forecast_PED = forecast['PED']

# Plot the forecast
data_ts['PED'].plot(label='actual')
forecast_PED.plot(label='forecast')
plt.legend()


In [ ]:
## TCN method - Univariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = TCNModel(
    input_chunk_length=13,
    output_chunk_length=12,
    n_epochs=500,
    dropout=0.1,
    dilation_base=2,
    weight_norm=True,
    kernel_size=5,
    num_filters=3,
    random_state=0,
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2[data2['TIME1'] >= '2019-09-10']
data2.set_index('TIME1', inplace=True)
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', 'PED')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)

# Fit the model for the n next days
model.fit(data_scaled, past_covariates=covariates)
forecast = model.predict(n=91, past_covariates=covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)

# Create a single trace for the entire dataset
trace_all = go.Scatter(
    x=np.concatenate([data_ts.time_index, forecast.time_index, data2.index]),
    y=np.concatenate([data_ts.univariate_values(), forecast.univariate_values(), data2['PED'].values]),
    mode='lines',
    name='All',
    line=dict(color='red'),
    showlegend=False
)

# Create separate traces for each segment with desired colors
trace1 = go.Scatter(
    x=data_ts.time_index, 
    y=data_ts.univariate_values(),
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

trace2 = go.Scatter(
    x=forecast.time_index, 
    y=forecast.univariate_values(),
    mode='lines',
    name='Imputed data',
    line=dict(color='red'),
)

trace3 = go.Scatter(
    x=data2.index,
    y=data2['PED'],
    mode='lines',
    line=dict(color='#636EFA'),
    showlegend=False,
)

# Create the layout
layout = go.Layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Pedestrian volume'),
)

# Create the figure and add the traces
fig = go.Figure(data=[trace_all, trace1, trace2, trace3], layout=layout)

# Update x-axis range
fig.update_xaxes(range=['2019-04-10', '2019-10-30'])

# Show the figure
fig.show()

In [ ]:
## TCN method - Multivariate imputation using darts

# Initialize the transformer and the model
transformer = Scaler()
model = TCNModel(
    input_chunk_length=13,
    output_chunk_length=12,
    n_epochs=500,
    dropout=0.1,
    dilation_base=2,
    weight_norm=True,
    kernel_size=5,
    num_filters=3,
    random_state=0,
)
# Prepare the data
data1 = data.copy()
data1=data1.reset_index()
data1['TIME1'] = pd.to_datetime(data1['TIME1'])
data1 = data1[data1['TIME1'] <= '2019-06-11']
data2 = data.copy()
data2=data2.reset_index()
data2['TIME1'] = pd.to_datetime(data2['TIME1'])
data2 = data2.drop_duplicates(subset='TIME1')
data_ts = TimeSeries.from_dataframe(data1, 'TIME1', ['PED', 'AQI', #'Daily Case Rate / 100,000',
                                                     'Average Temp', 'Precipitation'], fill_missing_dates=True, freq='D')
data_scaled = transformer.fit_transform(data_ts)

# Create year and month covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=data_ts.start_time(), freq=data_ts.freq_str, periods=len(data_ts)+91),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)

# Fit the model for n next days
model.fit(data_scaled, past_covariates=covariates)
forecast = model.predict(n=91, past_covariates=covariates)

# Re-scale the values to the original scale
forecast = transformer.inverse_transform(forecast)
forecast_PED = forecast['PED']

# Plot the forecast
data_ts['PED'].plot(label='actual')
forecast_PED.plot(label='forecast')
plt.legend()
